<a href="https://colab.research.google.com/github/salmon9/python/blob/main/circle2_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from matplotlib import pyplot as plt

In [2]:
class Model (nn.Module):
  def __init__(self):
      super().__init__()
      self.rnn = torch.nn.RNN(2, 20)
      self.fc = torch.nn.Linear(20, 2)
  def forward(self, x):
      output, h = self.rnn(x, 20)
      output = self.fc(output[:, -1])
      return output, h

In [8]:
def get_dataset(n):
    parameter = np.radians(np.random.randint(0,360,n)) #0〜360のランダム値 p:媒介変数
    x = np.cos(parameter) + np.random.normal(scale=0.01)
    y = np.sin(parameter) + np.random.normal(scale=0.01)

    dataset = [x,y]
    dataset = torch.FloatTensor(dataset)

    return torch.utils.data.TensorDataset(dataset)

In [9]:
data_number = 100000 #準備するデータの数
batch_size  = 1000 # 1つのミニバッチのデータの数
data_loader = torch.utils.data.DataLoader(get_dataset(data_number), batch_size=batch_size)

In [10]:
model = Model()
criterion = nn.MSELoss()
learning_rate = 1e-2
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
loss_log = [] # 学習状況のプロット用
epoch = 100
for t in range(epoch):
    for input in data_loader: # 1ミニバッチずつ計算
        optimizer.zero_grad() 
        
        output = model(input)

#        loss_x = loss_criterion(output_x, input_x)
#        loss_y = loss_criterion(output_y, input_y)

        loss = criterion(output,input)

        loss.backward()
        
        optimizer.step()
         

    print(t, loss.item())
    loss_log.append(loss.item())

TypeError: ignored

In [ ]:
##学習したモデルと関数を比較する

fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_aspect('equal')

p = np.radians(np.linspace(0,360,100))

plt.plot(np.cos(p),np.sin(p), label="circle", color="red")

p_model = torch.FloatTensor(p.reshape(100,1))

x_model, y_model = model(p_model)

x = x_model.detach().numpy().reshape(1,100)[0]
y = y_model.detach().numpy().reshape(1,100)[0] # プロット用に変換、detach()が必要

plt.scatter(x, y, label='model')
plt.legend()
plt.xlabel('x')
plt.ylabel('y')
plt.show()